In [ ]:
import sys
import os
import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation

%matplotlib inline
src_path = os.path.expanduser('~/source/')
sys.path.append(src_path)


In [ ]:
#greg tools
sys.path.append(src_path+'simtools')
sys.path.append(src_path+'simtools/infoenginessims')

from integrators import rkdeterm_eulerstoch
from dynamics import langevin_underdamped, langevin_overdamped
import simulation
from simprocedures import basic_simprocedures as sp
from simprocedures import running_measurements as rp
from simprocedures import trajectory_measurements as tp
import analysis
import analysis.running_quantities
import analysis.hists_1D
from infoenginessims.api import *

In [ ]:
#kyle tools
import kyle_tools as kt
from sus.protocol_designer import Potential, Protocol, System
from informational_states.measure import MeasurementDevice, Measurement, TrajectoryEnsemble

### These are the physical parameters that characterize the system

In [ ]:
from FQ_sympy_functions import DeviceParams
from bit_flip_sweep import set_systems


In [ ]:
Dev= DeviceParams()
Dev.change_vals({'I_minus':0})
triv_store, triv_comp = set_systems(Dev, comp_tau=10, d_store_comp=[.2,.2])
Dev.change_vals({'ell':Dev.ell*.885**2})
triv_store_2, triv_comp_2 = set_systems(Dev, comp_tau=10, d_store_comp=[.2,.2])

In [ ]:
triv_comp.protocol.params[:,0] 

In [ ]:
triv_store.protocol.params[:,0] 

In [ ]:

#params 1:
kT = .41*1.38E-23
C = 10*400E-12
R = 371
L = 10E-10

'''
#params 2:
kT = 6.9E-24
C = 530E-15
R = 2.1
L = 140E-12
'''

#these are some relevant dimensionful scales: alpha are the natural units for the JJ fluxes and U_0 is the natural scale for the potential
alpha = 2.07E-15 / (2*np.pi)
#IMPORTANT: all energies are measured in units of U_0
U_0 = alpha**2 / L
h = 6.63E-34

#these are important dimensionless simulation quantities, accounting for 
#m being measured in units of C, lambda in units of 1/R, energy in units of U_0
m_prime = np.array((1, 1/4))
lambda_prime = np.array((2, 1/2))
kT_prime = kT/U_0

print('Some Common Circuit Parameters')
Q = R*sqrt(C/L)
print( 'Q:{:.2f}'.format(Q))
frq = 1/(2*np.pi*sqrt(L*C))
print('f_LC in GHz:{:.2f}'.format(frq/1E9))
print('ring down in ns:{:.2f}'.format(1E9*Q/frq))
j_c = alpha/L
print('critical current density in mu_A:{:.2f}'.format(1E6*j_c))
print('Energy Scale')
print('U_0/kT:{:.2f}'.format(1/kT_prime))
print('kT/(h*f)',kT_prime*U_0 / (h*frq))


### First few cells are to set up the "system": the potential and the time dependent signal sent to its parameters

In [ ]:

#some sample systems for the bit flip, defined in the fq_systems.py file. You can use that file as a model for making new protocols:

from sus.library.fq_systems import fq_pot, flip_on, flip_off, flip_prot 

#equilibrating
eq_sys = System(fq_pot.trivial_protocol(), fq_pot)

#starts in EQ, holds the flip potential indefinitely
eternal_flip_sys = System(flip_on, fq_pot)

#starts in flip potential and then relaxes to EQ
diagnostic_flip_sys = System(flip_off, fq_pot)

#full flip, start in EQ and then end in EQ again
flip_sys = System(flip_prot, fq_pot)
alt_flip_sys = flip_sys.copy()
test_flip_sys = flip_sys.copy()



In [ ]:
eq_sys.protocol.params[:,0]

In [ ]:
fq_pot.default_params

In [ ]:
eternal_flip_sys.protocol.params

In [ ]:

test_flip_sys.protocol.protocols[0].params[1,:]=(-2.4,-2*np.pi)
test_flip_sys.protocol.protocols[1].params[1,:]=(-2*np.pi, -2.4)
#for asymetric device, add:
test_flip_sys.protocol.protocols[0].params[0,:]=(.069, .15)
test_flip_sys.protocol.protocols[1].params[0,:]=(.15, .069)

test_flip_sys.protocol.times[1,1]=3.72
test_flip_sys.protocol.refresh_substage_times()

In [ ]:

flip_sys.protocol.protocols[0].params[1,:]=(-2.4,-2.8)
flip_sys.protocol.protocols[1].params[1,:]=(-2.8, -2.4)
#for asymetric device, add:
flip_sys.protocol.protocols[0].params[0,:]=(.069, .15)
flip_sys.protocol.protocols[1].params[0,:]=(.15, .069)


In [ ]:
#this is a decent timescale for the current parameters found with a parameter sweep
flip_sys.protocol.times[1,1]=5.53
#flip_sys.protocol.times[1,1]=6.4
flip_sys.protocol.refresh_substage_times()

In [ ]:
# trying a device with different inductances and criticial currents
alt_flip_sys.protocol.protocols[0].params[1:4,:] = [[-2.9, -3.2],[27.51,27.51],[19.2,19.2]]
alt_flip_sys.protocol.protocols[1].params[1:4,:] = [[-3.2, -2.9],[27.51,27.51],[19.2,19.2]]

alt_flip_sys.protocol.protocols[0].params[0,:]=(.03891, .15)
alt_flip_sys.protocol.protocols[1].params[0,:]=(.15, .03891)
'''
alt_flip_sys.protocol.protocols[0].params[0,:]=(.069, .15)
alt_flip_sys.protocol.protocols[1].params[0,:]=(.15, .069)
'''
alt_flip_sys.protocol.times[:,0]=0, 1
alt_flip_sys.protocol.times[:,1]=1, 4.55
alt_flip_sys.protocol.refresh_substage_times()
alt_flip_sys.protocol.times

In [ ]:
flip_sys.protocol.protocols[0].params

In [ ]:
flip_sys.protocol.protocols[1].t_f

In [ ]:
#this cell defines what system you want to simulate and how many trials to run.
#generally no need to do lots of trials while prototyping protocols
N=10_000
system= flip_sys
eq_sys = system
system.mass= m_prime
eq_sys.mass = m_prime

system.potential.scale=1

#initialize the state in a rough EQ distribution.
initial_state = eq_sys.eq_state(N, t=0, beta=1/(kT_prime), manual_domain=[[-4,-4],[4,0]], axes=[1,2])

### Next few cells are visualization checks that your system is set up how you want

In [ ]:
#this cell checks for closeness to equilibrium, the true equilibrium dist will yield an identity matrix
#though you dont need true EQ for qualitative checks of behavior 
#and it takes a lot of trials to converge to identity
from sympy import Matrix
I = np.zeros((4,4))
for i in range(4):
    I[i,i] = 1

X = np.append(initial_state[...,0], system.mass * initial_state[...,1], axis=1)
d_H = np.append( -eq_sys.get_external_force(initial_state,0), initial_state[...,1], axis=1)
ept_test = np.einsum('in,im->inm',X, d_H)/kT_prime
Matrix(np.mean(ept_test, axis=0)-I)

In [ ]:
initial_state[is_bools['0']][...,0,0].std()

In [ ]:
#this cell is for checking that the initial state is what you want
nbins= 100
phi_hist = np.histogram(initial_state[:,0,0], bins=nbins)
phi_dc_hist = np.histogram(initial_state[:,1,0], bins=nbins)
v_phi_hist = np.histogram(initial_state[:,0,1], bins=nbins)
v_phi_dc_hist = np.histogram(initial_state[:,1,1], bins=nbins)


#change the type of histogram to look at different coordinates
analysis.hists_1D.plot_hist(phi_hist);

In [ ]:
is_bools.keys()

In [ ]:
is_bools = kt.separate_by_state(initial_state[...,0,0])
isz = initial_state[is_bools['0']]
iso = initial_state[is_bools['1']]
sum(is_bools['1'])/sum(is_bools['0'])
is_means = np.array( [item.mean(axis=0) for item in [isz,iso]])

In [ ]:
#initial_state = is_means
initial_state= np.append(initial_state,is_means, axis=0)

In [ ]:
for key,val in is_bools.items():
    is_bools[key] = np.append(val, [False,False] , axis=0)

In [ ]:
#gives a snapshot of the potential at some time in some domain
t_snapshot=0
system.show_potential(t_snapshot, manual_domain=[[-4,-4],[4,-1]], surface=False)

In [ ]:
system.show_potential(3, manual_domain=[[-4,-4],[4,-1]], surface=False)

### Now we set up the simulation parameters

In [ ]:
# this sets up our simulation to do langevin dynamics.
# if you want to change the temperature or damping by some amount, you can change the scale factors in this cell
# probably dont want to change anythign else in here though

#NOTE: changing the temperature here will not change the temperature used to generate the EQ distribution,
#NOTE: time is scaled in units of sqrt(LC)
gamma = (lambda_prime/m_prime) * np.sqrt(L*C) / (R*C) 
theta = 1/m_prime
eta = (L/(R**2 * C))**(1/4) * np.sqrt(kT_prime*lambda_prime) / m_prime        
 

damping_scale = 1
temp_scale = 1

gamma = np.multiply(gamma, damping_scale)
eta = np.multiply(eta, np.sqrt(damping_scale*temp_scale))

dynamic = langevin_underdamped.LangevinUnderdamped(theta, gamma, eta, system.get_external_force)
dynamic.mass = system.mass

integrator = rkdeterm_eulerstoch.RKDetermEulerStoch(dynamic)

In [ ]:
procedures = [
    sp.MeasureStepValue(rp.get_current_state, trial_request = np.s_[0], output_name='zmeans'),
    sp.MeasureStepValue(rp.get_current_state, trial_request = np.s_[1], output_name='omeans')
]


In [ ]:
#dont change this cell unless you take a look at how the procedures work, this should be fine for most use cases

procedures = [
              sp.ReturnFinalState(),
              sp.MeasureAllState(trial_request=slice(0, 1500), step_request=np.s_[::4]),
              sp.MeasureAllState(trial_request=np.s_[-2:], output_name='mean_evolution'),
              rp.MeasureAllValue(rp.get_dW, 'all_W'),
              rp.MeasureFinalValue(rp.get_dW, 'final_W'),
              sp.MeasureMeanValue(rp.get_kinetic, output_name='kinetic' ),
              sp.MeasureMeanValue(rp.get_potential, output_name='potential'),
              sp.MeasureMeanValue(rp.get_EPT, output_name='equipartition'),
              sp.MeasureMeanValue(rp.get_current_state, output_name = 'zero_means', trial_request=is_bools['0']),
              sp.MeasureMeanValue(rp.get_current_state, output_name = 'one_means', trial_request=is_bools['1']),
              tp.CountJumps(state_slice=np.s_[...,0,0])
             ]

In [ ]:
# here is where you choose the number of steps to simulate and how long to run the sim for.
# note that if your time is longer than the protocol time, the potential will just sit at its final value.

nsteps =1000

total_time = (system.protocol.t_f-system.protocol.t_i)+1

dt = total_time / nsteps

sim = simulation.Simulation(integrator.update_state, procedures, nsteps, dt,
                            initial_state)

sim.system = system

### This is running the actual sim

In [ ]:
%%time
sim.output = sim.run(verbose=True)

In [ ]:
# this is assinging variables to the different sim outputs
all_state = sim.output.all_state['states']
all_W = sim.output.all_W
final_W = sim.output.final_W
final_state = sim.output.final_state
all_EPT = sim.output.equipartition['values']
all_KE = sim.output.kinetic['values']
all_PE = sim.output.potential['values']
times = np.linspace(0, total_time, nsteps+1)
z_states = sim.output.zero_means['values']
z_err = sim.output.zero_means['std_error']
o_states = sim.output.one_means['values']
o_err = sim.output.one_means['std_error']
mean_evo = sim.output.mean_evolution['states']
jumps = sim.output.trajectories


In [ ]:
plt.plot(z_states[:,0,0])

In [ ]:
try: zmean_evo1 = sim.output.zmeans['states'].squeeze(axis=0)
except: pass
try: omean_evo1 = sim.output.omeans['states'].squeeze(axis=0)
except: pass

In [ ]:
zmean_evo1.shape

In [ ]:
#plt.plot(z_states[:,0,0,1].transpose());
#plt.plot(o_states[:,0,0,1].transpose());
#plt.plot(z_states2[:,0,1].transpose());
#plt.plot(o_states2[:,0,1].transpose());
#plt.plot(zmean_evo[:,0,1].transpose());
#plt.plot(omean_evo[:,0,1].transpose());
plt.plot(zmean_evo1[:,0,1].transpose());
plt.plot(omean_evo1[:,0,1].transpose());


In [ ]:
z_states.shape

### After running the sim, there are plenty of analysis tools

In [ ]:
#plotting the trajectories along a particular axis

end_plot_time = 1*total_time #* 1 / 100
trial_indices = np.s_[:1500]


analysis.running_quantities.plot_running_quantity(all_state[trial_indices,:,0,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='phi v t')
plt.grid(True, which='both')
analysis.running_quantities.plot_running_quantity(all_state[trial_indices,:,1,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='phi_dc v t')

plt.grid(True, which='both')

In [ ]:
end_plot_time = 1*total_time #* 1 / 100

plt.rcParams['font.size'] = 16
legendsize = 6
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

rc_dict = {'axes.labelsize':'large', 'ytick.right':False,'legend.loc':'upper right', 'legend.fontsize':'xx-small', 'figure.autolayout':True, 'figure.figsize': (6,4.5)}
for item in rc_dict:
    plt.rcParams[item] = rc_dict[item]

plt.rc('grid', linestyle="-", color='black')

fig, ax = plt.subplots(2, sharex=True)

ax[0].grid(True, which='both')
analysis.running_quantities.plot_running_quantity(all_state[trial_indices,:,0,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='', ax=ax[0], alpha=.3)

ax[1].grid(True, which='both')
analysis.running_quantities.plot_running_quantity(all_state[trial_indices,:,1,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='', ax=ax[1], alpha=.3)

ax[0].set_xlabel('')    
ax[1].set_xlabel('$t(\sqrt{LC}$)')
ax[0].set_ylabel('$\\varphi$')
ax[1].set_ylabel('$\\varphi_{dc}$')

tick_params = {'which':'major', 'labelsize':12, 'size':2, 'direction':'inout', 'width':.6}
for item in ax:
    item.axvline(system.protocol.t_f,c='k', linestyle='dashed')
    item.axvline(1,c='k', linestyle='dashed')
    item.tick_params(**tick_params)
    



In [ ]:
fig.savefig('all_state_alpha.pdf')

In [ ]:
pwd

In [ ]:
from FQ_sympy_functions import fidelity

fidelity(jumps)

In [ ]:
x = str(np.s_[::2])

In [ ]:
names = ['+ to -', '- to +']
tot_fails = 0
for i, key in enumerate(jumps):
    succ, tot = sum(jumps[key]==2), sum(jumps[key]!=0)
    print(names[i],' fidelity:{:.3f}'.format(succ/tot))
    tot_fails += tot-succ
print('overall:{:.3f}'.format(1-tot_fails/N))

In [ ]:
plt.plot(z_states[150:,0,1])

In [ ]:
from bit_flip_sweep import get_tau_candidate

tau_candidates, t_crit = get_tau_candidate(z_states[150:], o_states[150:], times[150:])

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(20,10))
indices=np.s_[150::]
for item in [z_states[indices, :, 0, :], o_states[indices,:,0,:]]:
    for item in [item[...,0], item[...,1]]:
        ax[0].errorbar(times[indices], item[:,0], yerr=sqrt(N)*item[:,1], linestyle=':', alpha=.6)

ax[0].plot(times[indices], mean_evo[0,indices,0,0],c='k')
ax[0].plot(times[indices], mean_evo[0,indices,0,1],c='k')
ax[0].plot(times[indices], mean_evo[1,indices,0,0],c='k')
ax[0].plot(times[indices], mean_evo[1,indices,0,1],c='k')

for item in [z_states[indices, :, 1, :], o_states[indices,:,1,:]]:
    for item in [item[...,0], item[...,1]]:
        ax[1].errorbar(times[indices], item[:,0], yerr=sqrt(N)*item[:,1], alpha=.2)

        
ax[1].plot(times[indices], mean_evo[0,indices,1,0], c='k')
ax[1].plot(times[indices], mean_evo[0,indices,1,1],c='k')
ax[1].plot(times[indices], mean_evo[1,indices,1,0],c='k')
ax[1].plot(times[indices], mean_evo[1,indices,1,1],c='k')

for item in ax:
    item.grid(True, which='both')
    item.axvline(system.protocol.t_f, color='k')
    item.axvline(tau_candidates[0], color='k')
    item.axvline(tau_candidates[1], color='k')
    item.axvline(t_crit, color='k')

 


In [ ]:
#z_displacement = (z_states[...,0,:,0] - z_states[0,0,0,0])
#o_displacement = (o_states[...,0,:,0] - o_states[0,0,0,0])

displacement = final_state[...,0]-initial_state[0,...,0]
scaled_var = np.var(displacement, axis=0)/(np.mean(displacement, axis=0)**2)
sigma = np.mean(final_W, axis=0)/kT_prime
print(2/(np.exp(sigma)-1), 2/sigma, scaled_var)


In [ ]:
#plot potential energy

times = np.linspace(0, total_time, nsteps+1)

fig, ax = plt.subplots(figsize=(15,5));
ax.plot(times, all_PE[:,0]-all_PE[0,0]);

#ax.axvline(3.72, color='r')
plt.rc('grid', linestyle="-", color='black')


ax.errorbar(times, all_KE[:,0]-all_KE[0,0], yerr=3*all_KE[:,1]);
ax.plot(times, all_KE[:,0])
ax.legend(['potential','kinetic'])
ax.set_title(' Potential Energy (U_0) vs time');
plt.grid(True, which='both')
ax.axvline(tau_candidates[0], color='k')
ax.axvline(tau_candidates[1], color='k')
ax.axvline(t_crit, color='k')
#ax.axvline(system.protocol.t_f, color='r')

In [ ]:
fig, ax = plt.subplots()
ax.plot(times[1200:], all_KE[1200:,0]+all_PE[1200:,0]-(all_KE[0,0]+all_PE[0,0]))
ax.axvline(tau_candidates[0], color='k')
ax.axvline(tau_candidates[1], color='k')
ax.axvline(t_crit, color='k')

In [ ]:
EPT[0,0].shape

In [ ]:
'''
#plot equipartition check, this is mostly for checking if the distribution is equilibrium
EPT = all_EPT/kT_prime
EPT[:,0], EPT[:,1]
trials = np.s_[:200]

fig, ax = plt.subplots(figsize=(15,5));
ax.errorbar(times[trials], EPT[trials,0,0], yerr=3*EPT[trials,1,0]);
ax.errorbar(times[trials], EPT[trials,0,1], yerr=3*EPT[trials,1,1]);
ax.set_title(' \'Equipartition Energy\' (k_B T) vs time');
ax.legend(['phi', 'phi_dc'])
'''

In [ ]:
%%capture

#these cells make am animation of a 2D slice of phase space trajectories. You can plot velocities by changing the zero in all_state[...,0] to a 1
ani_exp = kt.animate_sim(all_state[...,0], total_time, frame_skip=5, color_by_state=True, key_state=None, axes_names=['phi','phi_dc'])
HTML(ani_exp.to_jshtml(fps=20))

In [ ]:
ani_exp

In [ ]:
%%capture

#these cells make am animation of a 2D slice of phase space trajectories. You can plot velocities by changing the zero in all_state[...,0] to a 1
phase_ani = kt.animate_sim(all_state[...,0,:], total_time, color_by_state=True, frame_skip=5, key_state=all_state[:,0,0,0], axes_names=['phi','v_phi'])
HTML(ani_exp.to_jshtml(fps=20))

In [ ]:
phase_ani

In [ ]:
analysis.running_quantities.plot_running_quantity(all_W[:1000],
                                                  final_time=total_time,
                                                  end_plot_time=total_time)

In [ ]:
#this will show you a histogram of the net work, with the mean and +- sigma marked
#note the energy scale is in k_B T

final_W_hist = np.histogram(final_W/kT_prime, bins=50)
fig, ax = analysis.hists_1D.plot_hist(final_W_hist, log=True)
m=(final_W/kT_prime).mean()
s=(final_W/kT_prime).std()
print(m)
ax.axvline(m, color='k')
ax.axvline(m-3*s, color='k')
ax.axvline(m+3*s, color='k')